## Open AI API를 사용한 데이터 증강

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/raw/train.csv")
df.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0


In [3]:
import openai

# Set your OpenAI API key
client = openai.OpenAI(api_key="openai-api-key")

In [ ]:
# Make a request to the OpenAI API

# Add a new message to the conversation
from tqdm import tqdm
import json

gpt_df = pd.DataFrame(
    columns=["id", "source", "sentence_1", "sentence_2", "label", "binary-label"]
)

for i in tqdm(range(len(df))):
    chat_completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Make some modifications to the two sentences so that the meaning does not change.
Make the output in only json form with sentence1 and sentence2 as attributes.

exampleoutput: {{
  "sentence1": "first sentence.",
  "sentence2": "secnond sentence."
}}

sentence1: {df['sentence_1'].iloc[i]}
sentence2: {df['sentence_2'].iloc[i]}
""",
            }
        ],
    )
    # Print the response from the OpenAI API
    response_message = chat_completion.choices[0].message.content
    # Parse the response message as JSON
    try:
        response_json = json.loads(response_message)
        # Append the new sentences to the dataframe
        new_row = {
            "id": df["id"].iloc[i],
            "source": "openai",
            "sentence_1": response_json["sentence1"],
            "sentence_2": response_json["sentence2"],
            "label": df["label"].iloc[i],
            "binary-label": df["binary-label"].iloc[i],
        }
        gpt_df = pd.concat([gpt_df, pd.DataFrame([new_row])], ignore_index=True)
    except:
        print(f"{i}에서 오류 발생")
        continue

In [17]:
gpt_df.head()

,id,source,sentence_1,sentence_2,label,binary-label
9324,boostcamp-sts-v1-train-000,openai,긴장감도 있고 놀라운 반전도 있으며 다른 한국 영화들과는 질적으로 다르네요~,놀라운 반전과 사랑이 깃들어 있으며 재미도 있답니다.,2.2,0.0
9325,boostcamp-sts-v1-train-001,openai,"앗, 접근 권한이 없다고 표시됩니다;;","오, 액세스 권한이 없다고 합니다.",4.2,1.0
9326,boostcamp-sts-v1-train-002,openai,주택청약 조건을 수정해 주세요.,주택청약의 무주택 기준을 수정해 주세요.,2.4,0.0
9327,boostcamp-sts-v1-train-003,openai,입사 후 처음 얼굴을 마주 주셔서 기뻤습니다.,화상으로만 보다가 실제로 만나니 정말 기뻤습니다.,3.0,1.0
9328,boostcamp-sts-v1-train-004,openai,정말 기쁘네요!!,꼭 실제로 한 번 만나고 싶어요 뿌뿌뿌~!~!,0.0,0.0


In [21]:
# Concatenate gpt_df and df
combined_df = pd.concat([df, gpt_df], ignore_index=True)

# Save the combined dataframe to a CSV file
combined_df.to_csv("data/custom/train.csv", index=False)

In [19]:
# Initialize new columns with None
# Create a new DataFrame to store the updated sentences
updated_df = df.copy()

# Initialize new columns with None
updated_df["sentence_1_gpt"] = None
updated_df["sentence_2_gpt"] = None

# Create a dictionary for quick lookup
df_subset_dict = gpt_df.set_index("id")[["sentence_1", "sentence_2"]].to_dict("index")

# Update the new DataFrame with matching sentences from df_subset
for idx, row in updated_df.iterrows():
    if row["id"] in df_subset_dict:
        updated_df.at[idx, "sentence_1_gpt"] = df_subset_dict[row["id"]]["sentence_1"]
        updated_df.at[idx, "sentence_2_gpt"] = df_subset_dict[row["id"]]["sentence_2"]

updated_df.head()

,id,source,sentence_1,sentence_2,label,binary-label,sentence_1_gpt,sentence_2_gpt
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0,긴장감도 있고 놀라운 반전도 있으며 다른 한국 영화들과는 질적으로 다르네요~,놀라운 반전과 사랑이 깃들어 있으며 재미도 있답니다.
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0,"앗, 접근 권한이 없다고 표시됩니다;;","오, 액세스 권한이 없다고 합니다."
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0,주택청약 조건을 수정해 주세요.,주택청약의 무주택 기준을 수정해 주세요.
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0,입사 후 처음 얼굴을 마주 주셔서 기뻤습니다.,화상으로만 보다가 실제로 만나니 정말 기뻤습니다.
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0,정말 기쁘네요!!,꼭 실제로 한 번 만나고 싶어요 뿌뿌뿌~!~!


In [20]:
updated_df.sample(10)[["sentence_1", "sentence_1_gpt", "sentence_2", "sentence_2_gpt"]]

,sentence_1,sentence_1_gpt,sentence_2,sentence_2_gpt
4724,해외에서 동성결혼 해서 부부로 인정되는 경우 국내에서도 인정해주세요,"해외에서 동성결혼하여 부부로 인정받는 경우, 국내에서도 인정해 주시길 바랍니다.",이슬람이 공식종교라면 신천지 증산도 통일교도 공식종교로 만들어주세요.,"이슬람이 공식 종교라면 신천지, 증산도, 통일교도 공식 종교로 인정해 주세요."
6229,어디든 기댈곳이 없다.,어디에서도 의지할 곳이 없다.,재정신으로는 절대 만들 수 없다.,정신적으로는 결코 만들 수 없다.
8701,대통령 문제가 있습니다,대통령에 관한 문제가 있습니다.,서울시 차량2부제 문제 있습니다,서울시의 차량 2부제에 대한 문제가 있습니다.
3370,기상청을 해체시켜주세요.,기상청을 없애주세요.,금융위원회를 해체해주세요,금융위원회를 없애주세요.
5248,평가인증 때문에 너무 힘들어요,평가인증으로 인해 너무 힘들어요.,평가인증 때문에 너무 힘들어,평가인증으로 인해 너무 힘들어.
3412,다띄워놓고 돌려도 아직도 램이 남아요.,다 띄워놓고 돌려도 아직도 램이 남아있어요.,단행본이 아직 완결이 안 되었네요.. 그래도 사놓을게요 ㅋ,단행본이 아직 마무리되지 않았군요.. 그래도 구매할게요 ㅋ
7189,"시시한 전개, 시시한 결말, 촌스런 OOO전 영화.","별로인 전개, 별로인 결말, 촌스러운 OOO 영화.","지루한 전개, 지루한 결말, 엉성한 OOO 영화.","따분한 전개, 따분한 결말, 어설픈 OOO 영화."
6423,도전하실 바당요가도 엎어지면 코 닿을 거리에 있습니다,도전하실 바당 요가는 엎어지면 코 닿을 거리입니다.,같이 도전하실 분을 한 번 찾아봅니다!,함께 도전할 분을 한 번 찾아보겠습니다!
8026,그래도 후하진 못해..,그래도 후한 건 아니야..,그래도 잘 만들었네...,그래도 잘 만들어냈네...
4360,"주4일제! 단, 조건이 있습니다","주 4일제! 단, 조건이 있습니다.","주 4일! 단, 조건이 있다",주 4일제입니다! 그러나 조건이 있습니다.


End.